In [1]:
import re
import json
import time
import requests
import os.path
import warnings

### Preprocessing

In [2]:
!pip install jieba_fast
!pip install emoji

     |████████████████████████████████| 7.5 MB 7.9 MB/s 
  Created wheel for jieba-fast: filename=jieba_fast-0.53-cp37-cp37m-linux_x86_64.whl size=7658320 sha256=6c3c0585e20022be004abd1a0ccf8e9ce441b4ea55495cf15fb330b411637e95
  Stored in directory: /root/.cache/pip/wheels/40/87/79/51f40efba790e648b37d3f3a7c03fe0b2c93973d9ed0076bae
Successfully built jieba-fast
     |████████████████████████████████| 175 kB 4.7 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=9b4e6cf2f06eacc2f5bfb65221ff57ef8dc50896ff102f87107af3254ceab9ee
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [3]:
import jieba_fast
import pandas as pd
import re
import emoji

In [5]:
#load data by mongodb
# comms = []
# for item  in music_db.find():
#     comms.append(item)
    
# len(comms)
#load data by json

with open('datasets.json','r', encoding='utf-8') as f:
    comms = json.loads(f.read())

    
len(comms)

31722

In [6]:
# save to json
#with open('datasets.json','w',encoding='utf-8') as w:
#    w.write(json.dumps(comms,ensure_ascii=False))
    

In [7]:
# to DataFrame
df = pd.json_normalize(comms)
df

,_id,nike,comment,praisenum,time,m_name,tag,topid
0,1!PeUgh6.tYeG1pnsFPaL4RxV8qkil0qUWuMdjDoNh1l.p...,性格决定命运,我特别喜欢这首歌，我现在12岁了，基本每天都听[em]e400408[/em][em]e40...,1650,2016-07-16 21:17:25,我爱洗澡,pos,451197
1,1!L3BkyxeMbghZ3vfKIPYB9EhtdxcCT-oHYPumB3wpaUE*,半心~,最近孩子睡前的歌来自我的第一个追星偶像范晓萱小魔女时代的歌曲，当时五六年级的我每一首歌词每一...,830,1971-05-27 08:08:03,我爱洗澡,pos,451197
2,1!tw8vlS87r7gnvU3Y7Zs4UpqL0rke2pViLn6tNAHtJvE*,失爱！,有多少人和我一样做什么事时脑子里是有默认音乐伴奏的～～比如一个人逛街就自动回响起林忆莲的站在...,606,1971-05-27 08:13:54,我爱洗澡,pos,451197
3,1!8PUpWiGR9.m7sx0hW3xQT5phubZyaiBrtBs8lGMVmgZT...,南一,室友洗澡太慢了，我每次都是拿一个蓝牙音箱发门口发这个歌，从此以后他洗澡好快的[em]e400...,589,2017-12-30 23:39:55,我爱洗澡,pos,451197
4,1!PeUgh6.tYeG1pnsFPaL4R0s3VbEVL2tuwIEjAJlj2a6V...,,好好听，我从小不爱洗澡，听了这首歌，我就爱上洗澡了，[em]e400832[/em][em]...,492,2016-06-17 22:26:01,我爱洗澡,pos,451197
...,...,...,...,...,...,...,...,...
31717,1!wQ0GvmNhQkbhHzUanYqnVw3Sn7ZsJ2FGzCQLvmabQnq....,茗千.,追凌！追凌！追凌！听着听着姨母笑，诶嘿嘿(๑´ㅂ`๑),18,2018-07-31 10:47:48,爱啦啦,pos,102076884
31718,1!8GfUcY9CJj3M-I99cTUB5gamh3R355q-fwYIJL9tOBpo...,也Yeah,发现了好家伙这变声器真不错,18,2021-02-25 23:00:55,爱啦啦,pos,102076884
31719,1!8GfUcY9CJj3M-I99cTUB5mtDa0N6SxiGcPunhLNpr3Ua...,斗鱼肯德鸭,大班听的歌，现在已经初一了[em]e400101[/em][em]e400408[/em],19,2021-09-22 19:50:17,爱啦啦,pos,102076884
31720,1!8GfUcY9CJj3M-I99cTUB5uNEo6grzE0c.W7gv2Ib91QI...,神仙.,从光热大法来的！！！这个坑是出不去了！！甜死我了！！！[em]e400831[/em][em...,17,2020-03-01 16:28:57,爱啦啦,pos,102076884


In [8]:
pattern = r"[!\"#$%&'()*+,-./:;<=>?@[\\\]^_^{|}~—！，。？、￥…（）：【】《》‘’“”\s]+"
re_obj = re.compile(pattern)


def clear(text):
    pattern = r"[!\"#$%&'()*+,-./:;<=>?@[\\\]^_^{|}~—！，。？、￥…（）：【】《》‘’“”\s]+"
    re_obj = re.compile(pattern)

    return re_obj.sub("", text)


def filter_ele(text):

    pattern = re.compile(r'(\[em\])(.*)(\[\/em\])')
    return pattern.sub('', text)


def re_emoji(text):
    text = emoji.demojize(text)
    result = re.sub(':\S+?:', ' ', text)
    return result


def filter_str(text):
    res = re.compile("[^\\u4e00-\\u9fa5^a-z^A-Z^0-9]")
    return res.sub('', text)


def filter_emoji(text):
    res = re.compile(u'[\U00010000-\U0010ffff\\uD800-\\uDBFF\\uDC00-\\uDFFF]')
    return res.sub('', text)



df['comment_clear'] = df['comment'].apply(filter_ele)

# df['comment_clear'] = df['comment_clear'].apply(filter_emoji)
df['comment_clear'] = df['comment_clear'].apply(re_emoji)
df['comment_clear'] = df['comment_clear'].apply(clear)
df['comment_clear'] = df['comment_clear'].apply(filter_str)

df['comment_clear']

0                                    我特别喜欢这首歌我现在12岁了基本每天都听
1        最近孩子睡前的歌来自我的第一个追星偶像范晓萱小魔女时代的歌曲当时五六年级的我每一首歌词每一个...
2        有多少人和我一样做什么事时脑子里是有默认音乐伴奏的比如一个人逛街就自动回响起林忆莲的站在这里...
3                     室友洗澡太慢了我每次都是拿一个蓝牙音箱发门口发这个歌从此以后他洗澡好快的
4                                   好好听我从小不爱洗澡听了这首歌我就爱上洗澡了
                               ...                        
31717                                     追凌追凌追凌听着听着姨母笑诶嘿嘿
31718                                        发现了好家伙这变声器真不错
31719                                         大班听的歌现在已经初一了
31720                                  从光热大法来的这个坑是出不去了甜死我了
31721                                              光热大旗永不倒
Name: comment_clear, Length: 31722, dtype: object

In [10]:
#stopword
with open('stop_words.txt','r', encoding='utf-8') as f:
    stopwords = f.readlines()
    stopwords = list(map(lambda x: x.strip(), stopwords))

def cut(text):
    return ' '.join(list(jieba_fast.cut(text)))

df['comment_cut'] = df['comment_clear'].apply(cut)
df['comment_cut']

0                       我 特别 喜欢 这 首歌 我 现在 12 岁 了 基本 每天 都 听
1        最近 孩子 睡前 的 歌 来自 我 的 第一个 追星 偶像 范晓萱 小 魔女 时代 的 歌曲...
2        有 多少 人 和 我 一样 做 什么 事时 脑子里 是 有 默认 音乐 伴奏 的 比如 一个...
3        室友 洗澡 太慢 了 我 每次 都 是 拿 一个 蓝牙 音箱 发 门口 发 这个 歌 从此以...
4                    好好 听 我 从小 不爱 洗澡 听 了 这 首歌 我 就 爱 上 洗澡 了
                               ...                        
31717                           追凌 追凌 追凌 听 着 听 着 姨母 笑 诶 嘿嘿
31718                                  发现 了 好家伙 这 变声 器 真不错
31719                                   大 班 听的歌 现在 已经 初一 了
31720                     从 光热 大法 来 的 这个 坑 是 出不去 了 甜 死 我 了
31721                                            光热 大旗 永不倒
Name: comment_cut, Length: 31722, dtype: object

### Train test split

In [11]:
from sklearn.model_selection import train_test_split
X= df['comment_cut']
y = df['tag']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Naive Bayes

In [64]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer



#bag of word

vec = CountVectorizer(max_df = 0.8, 
                       min_df = 3, 
                       token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b',
#                       max_features=5000
                       )

train_vec = pd.DataFrame(vec.fit_transform(X_train).toarray(), columns=vec.get_feature_names())
# TF-IDF feature
tfidf_vec = TfidfVectorizer(max_df=0.80, min_df=3)

In [65]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline

from sklearn.model_selection import GridSearchCV

NB = MultinomialNB(alpha=0.001)
X_train_vec = vec.fit_transform(X_train)
NB.fit(X_train_vec, y_train)
train_acc = NB.score(X_train_vec, y_train)
print(train_acc)

0.7906327403737897


In [66]:
nb_model = MultinomialNB()

clf = pipeline.Pipeline([('nb', nb_model)])

param_grid = {'nb__alpha': [0.001, 0.01, 0.1, 0.1, 0.5, 0.6, 1, 2, 5]}

model = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    n_jobs=3,
    refit=True,
    cv=5)


model.fit(X_train_vec, y_train)

GridSearchCV(cv=5, estimator=Pipeline(steps=[('nb', MultinomialNB())]),
             n_jobs=3,
             param_grid={'nb__alpha': [0.001, 0.01, 0.1, 0.1, 0.5, 0.6, 1, 2,
                                       5]})

In [67]:
model.fit(X_train_vec, y_train)

print("Best parameters :")
best_parameters = model.best_estimator_.get_params()

for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))


Best parameters :
	nb__alpha: 2


In [68]:
from sklearn.metrics import classification_report
train_pre = NB.predict(X_train_vec)
train_report  =classification_report(y_train, train_pre)
print(train_report)

              precision    recall  f1-score   support

         neg       0.78      0.76      0.77     10189
         pos       0.80      0.81      0.81     12016

    accuracy                           0.79     22205
   macro avg       0.79      0.79      0.79     22205
weighted avg       0.79      0.79      0.79     22205



In [69]:
X_test_vec = vec.transform(X_test)
print(NB.score(X_test_vec, y_test))

0.7028475359882316


In [70]:
test_pre = NB.predict(X_test_vec)
test_report  =classification_report(y_test, test_pre)
print(test_report)

              precision    recall  f1-score   support

         neg       0.68      0.67      0.67      4412
         pos       0.72      0.74      0.73      5105

    accuracy                           0.70      9517
   macro avg       0.70      0.70      0.70      9517
weighted avg       0.70      0.70      0.70      9517



# SVM

In [71]:

from sklearn import svm
from sklearn import metrics
import numpy as np
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier


In [72]:
vec = CountVectorizer(max_df = 0.8, 
                       min_df = 3, 
                       token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b',
#                       max_features=5000
                       )

In [73]:
pipeline = Pipeline([
        ('vect', TfidfVectorizer(min_df=3, max_df=0.95)),
        ('clf', LinearSVC(C=1000)),
    ])

In [74]:
parameters = {
        'vect__ngram_range': [(1, 1), (1, 2)],
    }
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1)
tfidf_train_vec = tfidf_vec.fit(X_train)
grid_search.fit(X_train, y_train)

grid_search.best_params_

{'vect__ngram_range': (1, 2)}

In [75]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import make_pipeline
# from sklearn.externals import joblib
tfidf_vectorizer = TfidfVectorizer(max_df=0.80, min_df=2)


Svm = SGDClassifier(
    penalty='l2',
    alpha=0.0001,
    l1_ratio=0.15,
    fit_intercept=True,
    max_iter=1000,
    tol=0.001,
    shuffle=True,
    verbose=0,
    epsilon=0.1,
)
pipe = make_pipeline(tfidf_vectorizer, Svm)
pipe.fit(X_train, y_train)
pipe.score(X_train, y_train)  # SVM accuracy

0.7928844854762441

In [76]:

train_pre = pipe.predict(X_train)
c_report = classification_report(y_train, train_pre)

print(pipe.score(X_train, y_train))
print(c_report)

0.7928844854762441
              precision    recall  f1-score   support

         neg       0.82      0.71      0.76     10189
         pos       0.78      0.87      0.82     12016

    accuracy                           0.79     22205
   macro avg       0.80      0.79      0.79     22205
weighted avg       0.80      0.79      0.79     22205



In [77]:

test_pre = pipe.predict(X_test)
c_report = classification_report(y_test, test_pre)

print(pipe.score(X_test, y_test))
print(c_report)

0.697488704423663
              precision    recall  f1-score   support

         neg       0.71      0.59      0.64      4412
         pos       0.69      0.79      0.74      5105

    accuracy                           0.70      9517
   macro avg       0.70      0.69      0.69      9517
weighted avg       0.70      0.70      0.69      9517



# Logistic Regression  Classifier

In [78]:
# Logistic Regression Classifier
tfidf_vectorizer = TfidfVectorizer(max_df=0.80, min_df=2)

Lr = LogisticRegression(C=1.0,)

pipe = make_pipeline(tfidf_vectorizer, Lr)
pipe.fit(X_train, y_train)
pipe.score(X_train, y_train)  # LR accuracy

0.8069804098176087

In [79]:
train_pre = pipe.predict(X_train)
c_report = classification_report(y_train, train_pre)

print(pipe.score(X_train, y_train))
print(c_report)

0.8069804098176087
              precision    recall  f1-score   support

         neg       0.82      0.74      0.78     10189
         pos       0.80      0.86      0.83     12016

    accuracy                           0.81     22205
   macro avg       0.81      0.80      0.80     22205
weighted avg       0.81      0.81      0.81     22205



In [80]:
test_pre = pipe.predict(X_test)
c_report = classification_report(y_test, test_pre)

print(pipe.score(X_test, y_test))
print(c_report)

0.701061258800042
              precision    recall  f1-score   support

         neg       0.70      0.62      0.66      4412
         pos       0.70      0.77      0.74      5105

    accuracy                           0.70      9517
   macro avg       0.70      0.70      0.70      9517
weighted avg       0.70      0.70      0.70      9517



# Random Forest Classifier 


In [81]:
# Random Forest Classifier --- gini index
tfidf_vectorizer = TfidfVectorizer(max_df=0.80, min_df=2)
Rf = RandomForestClassifier(
    n_estimators=100,
    criterion='gini',
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='auto',
    min_impurity_decrease=0.0,
)

# Lr = LogisticRegression(C=1.0,)

pipe = make_pipeline(tfidf_vectorizer, Rf)
pipe.fit(X_train, y_train)
pipe.score(X_train, y_train)  # RF accuracy

0.9876604368385499

In [82]:
#train
train_pre = pipe.predict(X_train)
c_report = classification_report(y_train, train_pre)

print(pipe.score(X_train, y_train))
print(c_report)

0.9876604368385499
              precision    recall  f1-score   support

         neg       0.99      0.98      0.99     10189
         pos       0.99      0.99      0.99     12016

    accuracy                           0.99     22205
   macro avg       0.99      0.99      0.99     22205
weighted avg       0.99      0.99      0.99     22205



In [83]:
#test
test_pre = pipe.predict(X_test)
c_report = classification_report(y_test, test_pre)

print(pipe.score(X_test, y_test))
print(c_report) # Rf Performs really good on train, but not so good on test. Overfitting.

0.678575181254597
              precision    recall  f1-score   support

         neg       0.66      0.62      0.64      4412
         pos       0.69      0.73      0.71      5105

    accuracy                           0.68      9517
   macro avg       0.68      0.67      0.68      9517
weighted avg       0.68      0.68      0.68      9517



In [84]:
# Random Forest Classifier --- entropy
tfidf_vectorizer = TfidfVectorizer(max_df=0.80, min_df=2)
Rf = RandomForestClassifier(
    n_estimators=100,
    criterion='entropy',
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='auto',
    min_impurity_decrease=0.0,
)

# Lr = LogisticRegression(C=1.0,)

pipe = make_pipeline(tfidf_vectorizer, Rf)
pipe.fit(X_train, y_train)
pipe.score(X_train, y_train)  # RF accuracy




0.9876604368385499

In [85]:
#train
train_pre = pipe.predict(X_train)
c_report = classification_report(y_train, train_pre)

print(pipe.score(X_train, y_train))
print(c_report)

0.9876604368385499
              precision    recall  f1-score   support

         neg       0.99      0.98      0.99     10189
         pos       0.99      0.99      0.99     12016

    accuracy                           0.99     22205
   macro avg       0.99      0.99      0.99     22205
weighted avg       0.99      0.99      0.99     22205



In [86]:
#test
test_pre = pipe.predict(X_test)
c_report = classification_report(y_test, test_pre)

print(pipe.score(X_test, y_test))
print(c_report) 

0.6872964169381107
              precision    recall  f1-score   support

         neg       0.68      0.62      0.65      4412
         pos       0.69      0.74      0.72      5105

    accuracy                           0.69      9517
   macro avg       0.69      0.68      0.68      9517
weighted avg       0.69      0.69      0.69      9517



# LSTM


In [88]:
#preprocessed
df_csv = pd.DataFrame(df['comment_clear'])
df_csv['label'] = df['tag'].apply(lambda x: 1 if x=='pos' else 0)
df_csv.to_csv('data.csv',index=False,encoding='utf_8_sig')

#unpreprocessed
df_csv = pd.DataFrame(df['comment'])
df_csv['label'] = df['tag'].apply(lambda x: 1 if x=='pos' else 0)
df_csv.to_csv('raw_data.csv',index=False,encoding='utf_8_sig')




In [ ]:
import pickle
from keras.models import load_model

import numpy as np
import pandas as pd
from keras.layers import LSTM, Dense, Embedding, Dropout
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils, plot_model
from sklearn.metrics import accuracy_score,classification_report
from sklearn.model_selection import train_test_split


# load data
def load_data(filepath, input_shape=20):
    df = pd.read_csv(filepath, encoding='utf_8_sig')
    ck1 =  df.columns[0]
    df[ck1] = df[ck1].apply(lambda x:str(x))

    labels, vocabulary = list(df['label'].unique()), list(df[ck1].unique())


    string = ''
    for word in vocabulary:
        string += str(word)

    vocabulary = set(string)


    word_dictionary = {word: i + 1 for i, word in enumerate(vocabulary)}
    with open('word_dict.pk', 'wb') as f:
        pickle.dump(word_dictionary, f)
    inverse_word_dictionary = {i + 1: word for i, word in enumerate(vocabulary)}
    label_dictionary = {label: i for i, label in enumerate(labels)}
    with open('label_dict.pk', 'wb') as f:
        pickle.dump(label_dictionary, f)
    output_dictionary = {i: labels for i, labels in enumerate(labels)}

    vocab_size = len(word_dictionary.keys())  
    label_size = len(label_dictionary.keys())  


    x = [[word_dictionary[word] for word in sent] for sent in df[ck1]]
    x = pad_sequences(maxlen=input_shape, sequences=x, padding='post', value=0)
    y = [[label_dictionary[sent]] for sent in df['label']]
    y = [np_utils.to_categorical(label, num_classes=label_size) for label in y]
    y = np.array([list(_[0]) for _ in y])

    return x, y, output_dictionary, vocab_size, label_size, inverse_word_dictionary



def create_LSTM(n_units, input_shape, output_dim, filepath):
    x, y, output_dictionary, vocab_size, label_size, inverse_word_dictionary = load_data(filepath)
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size + 1, output_dim=output_dim,
                        input_length=input_shape, mask_zero=True))
    model.add(LSTM(n_units, input_shape=(x.shape[0], x.shape[1])))
    model.add(Dropout(0.2))
    model.add(Dense(label_size, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    plot_model(model, to_file='./model_lstm.png', show_shapes=True)
    model.summary()

    return model


def model_train(input_shape, filepath, model_save_path):
    x, y, output_dictionary, vocab_size, label_size, inverse_word_dictionary = load_data(filepath, input_shape)
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3, random_state=42)

    n_units = 100
    batch_size = 32
    epochs = 5
    output_dim = 20


    lstm_model = create_LSTM(n_units, input_shape, output_dim, filepath)
    lstm_model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=1)


    lstm_model.save(model_save_path)
    # lstm_model = load_model(model_save_path)
    N = train_x.shape[0]  
    predict = []
    label = []
    for start, end in zip(range(0, N, 1), range(1, N + 1, 1)):
        sentence = [inverse_word_dictionary[i] for i in train_x[start] if i != 0]
        y_predict = lstm_model.predict(train_x[start:end])
        label_predict = output_dictionary[np.argmax(y_predict[0])]
        label_true = output_dictionary[np.argmax(train_y[start:end])]
        # print(''.join(sentence), label_true, label_predict)  
        predict.append(label_predict)
        label.append(label_true)

    acc = accuracy_score(label, predict)  
    print('训练集上的准确率为: %s.' % acc)
    print(classification_report(label, predict))

    N = test_x.shape[0]  
    predict = []
    label = []
    for start, end in zip(range(0, N, 1), range(1, N + 1, 1)):
        sentence = [inverse_word_dictionary[i] for i in test_x[start] if i != 0]
        y_predict = lstm_model.predict(test_x[start:end])
        label_predict = output_dictionary[np.argmax(y_predict[0])]
        label_true = output_dictionary[np.argmax(test_y[start:end])]
        # print(''.join(sentence), label_true, label_predict)  
        predict.append(label_predict)
        label.append(label_true)

    acc = accuracy_score(label,predict)  
    print('测试集上的准确率为: %s.' % acc)
    print(classification_report(label,predict))


if __name__ == '__main__':
    filepath = './raw_data.csv'
    input_shape = 300
    model_save_path = './raw_data_model_300.h5'
    model_train(input_shape, filepath, model_save_path)
    # lstm_model = load_model(model_save_path)
#

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 20)           109100    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               48400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 157,702
Trainable params: 157,702
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
22205/22205 [==============================] - 229s 10ms/step - loss: 0.6481 - accuracy: 0.6242
Epoch 2/5
22205/22205 [==============================] - 238s 11ms/step - loss: 0.5924 - accuracy: 0.6887
Epoch 3/5
22205/22205 [===========

In [ ]:


import pickle
from keras.models import load_model

import numpy as np
import pandas as pd
from keras.layers import LSTM, Dense, Embedding, Dropout
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils, plot_model
from sklearn.metrics import accuracy_score,classification_report
from sklearn.model_selection import train_test_split



def load_data(filepath, input_shape=20):
    df = pd.read_csv(filepath, encoding='utf_8_sig')
    ck1 =  df.columns[0]
    df[ck1] = df[ck1].apply(lambda x:str(x))

    labels, vocabulary = list(df['label'].unique()), list(df[ck1].unique())


    string = ''
    for word in vocabulary:
        string += str(word)

    vocabulary = set(string)


    word_dictionary = {word: i + 1 for i, word in enumerate(vocabulary)}
    with open('word_dict.pk', 'wb') as f:
        pickle.dump(word_dictionary, f)
    inverse_word_dictionary = {i + 1: word for i, word in enumerate(vocabulary)}
    label_dictionary = {label: i for i, label in enumerate(labels)}
    with open('label_dict.pk', 'wb') as f:
        pickle.dump(label_dictionary, f)
    output_dictionary = {i: labels for i, labels in enumerate(labels)}

    vocab_size = len(word_dictionary.keys())  
    label_size = len(label_dictionary.keys())  


    x = [[word_dictionary[word] for word in sent] for sent in df[ck1]]
    x = pad_sequences(maxlen=input_shape, sequences=x, padding='post', value=0)
    y = [[label_dictionary[sent]] for sent in df['label']]
    y = [np_utils.to_categorical(label, num_classes=label_size) for label in y]
    y = np.array([list(_[0]) for _ in y])

    return x, y, output_dictionary, vocab_size, label_size, inverse_word_dictionary




def create_LSTM(n_units, input_shape, output_dim, filepath):
    x, y, output_dictionary, vocab_size, label_size, inverse_word_dictionary = load_data(filepath)
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size + 1, output_dim=output_dim,
                        input_length=input_shape, mask_zero=True))
    model.add(LSTM(n_units, input_shape=(x.shape[0], x.shape[1])))
    model.add(Dropout(0.2))
    model.add(Dense(label_size, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    plot_model(model, to_file='./model_lstm.png', show_shapes=True)
    model.summary()

    return model


def model_train(input_shape, filepath, model_save_path):

    x, y, output_dictionary, vocab_size, label_size, inverse_word_dictionary = load_data(filepath, input_shape)
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3, random_state=42)


    n_units = 100
    batch_size = 32
    epochs = 5
    output_dim = 20


    lstm_model = create_LSTM(n_units, input_shape, output_dim, filepath)
    lstm_model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=1)


    lstm_model.save(model_save_path)
    # lstm_model = load_model(model_save_path)
    N = train_x.shape[0]  
    predict = []
    label = []
    for start, end in zip(range(0, N, 1), range(1, N + 1, 1)):
        sentence = [inverse_word_dictionary[i] for i in train_x[start] if i != 0]
        y_predict = lstm_model.predict(train_x[start:end])
        label_predict = output_dictionary[np.argmax(y_predict[0])]
        label_true = output_dictionary[np.argmax(train_y[start:end])]
        # print(''.join(sentence), label_true, label_predict)  
        predict.append(label_predict)
        label.append(label_true)

    acc = accuracy_score(label, predict)  
    print('训练集上的准确率为: %s.' % acc)
    print(classification_report(label, predict))

    N = test_x.shape[0]  
    predict = []
    label = []
    for start, end in zip(range(0, N, 1), range(1, N + 1, 1)):
        sentence = [inverse_word_dictionary[i] for i in test_x[start] if i != 0]
        y_predict = lstm_model.predict(test_x[start:end])
        label_predict = output_dictionary[np.argmax(y_predict[0])]
        label_true = output_dictionary[np.argmax(test_y[start:end])]
        # print(''.join(sentence), label_true, label_predict)  
        predict.append(label_predict)
        label.append(label_true)

    acc = accuracy_score(label,predict)
    print('测试集上的准确率为: %s.' % acc)
    print(classification_report(label,predict))


if __name__ == '__main__':
    filepath = './raw_data.csv'
    input_shape = 300
    model_save_path = './data_model_300.h5'
    model_train(input_shape, filepath, model_save_path)
    # lstm_model = load_model(model_save_path)
#

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 300, 20)           109100    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               48400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 157,702
Trainable params: 157,702
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
22205/22205 [==============================] - 258s 12ms/step - loss: 0.6516 - accuracy: 0.6227
Epoch 2/5
22205/22205 [==============================] - 277s 12ms/step - loss: 0.5997 - accuracy: 0.6846
Epoch 3/5
22205/22205 [===========